# Easy Tile Grid Maps with Python and Plotly
### A reusable template for a popular infographic
### by Lee Vaughan

A tile grid map in the shape of the USA is composed of red and blue squares representing states. The colors represent the 2024 US presidential election results, with blue being won by Harris and red by Trump.
Tile grid map of 2024 US presidential election results (Red=Trump; Blue=Harris) (by author)
Have you ever wondered how professional data journalists craft those clean, instantly understandable maps that anchor major stories? Their secret is often the tile grid map. This technique creates memorable graphics that make data pop.

A tile grid map is a data visualization that transforms irregular geographic shapes into uniform, equally sized tiles — generally squares or hexagons. This approach is favored by leading news outlets such as The Economist, The New York Times, and The Washington Post when visual clarity and balance are more important than strict geographic detail.

Tile grid maps serve a similar purpose to choropleth maps, where states (or other geographical regions) are colored or shaded to represent a unique statistical value, such as population. A drawback to US choropleth maps is that large states visually dominate the plot, and small states can be lost altogether.

A choropleth map of the USA with states colored in greens and browns denoting energy costs per state.
A choropleth map of energy costs is visually dominated by large states (Source: Wikimedia Commons)
By replacing traditional choropleth maps with a grid of consistent shapes, tile grid maps eliminate visual imbalances that can distract from the underlying data. The streamlined design also promotes quick, at-a-glance comprehension, even when publishing constraints limit available space.

Data journalists frequently employ tile grid maps to emphasize comparisons and identify regional patterns. This includes electoral maps and similar infographics where maintaining a simple and uniform visual structure is key.

Here’s an example addressing abortion access. Notice how tiny Rhode Island carries as much weight as mighty Texas:

A tile grid map composed of red, green, and orange squares representing US states shows abortion access levels in the USA. Green denotes legal as per Roe vs Wade, Orange is legal with some limits, and Red is banned.
Tile grid map of abortion access (by author)
In this Quick Success Data Science project, I provide a ready-to-use template for generating US tile grid maps, with examples using Python and the Plotly graphing library. This guide not only walks you through the process but also demonstrates how harnessing the power of simple visuals can elevate your data storytelling.

The Template
The tile grid template is a CSV file with columns for the state name, state abbreviation, and X and Y coordinates. The coordinates aren’t geospatial but Cartesian, with an origin point at the lower left.

A view of a CSV file with 4 columns.
The top 15 rows of states_square_tile_template.csv (by author)
Later in the code, we’ll use the X and Y values as the centers of squares representing each state. Then we’ll annotate the squares using the “Abbr” (abbreviation) column values. Here’s how it looks with a reference grid displayed:

A tile grid map composed of squares in the rough shape of the USA is displayed over a Cartesian coordinate grid system. Each square represents a single state.
The template with its reference grid (by author)
You can download the file (states_square_tile_template.csv) from this Gist. Use it by adding a column representing your data values for each state (the states are sorted alphabetically, without the District of Columbia (DC)).

Next, we’ll look at two examples — one categorical and one continuous — that show you how to apply the template.

Third-party Libraries
Besides Python, you’ll need to install Plotly and pandas. Plotly makes it easy to create tile grid maps and includes a dynamic “hover” feature that launches a pop-up window of additional data whenever the cursor is placed over a tile.

Plotly is designed to work with pandas, Python’s most popular data analysis library. You can install both at the same time using pip:

pip install plotly pandas

At the time of this writing, I used pandas version 2.0.3 and Plotly version 5.24.1.

Categorical Code
The following code example creates a tile grid map of categorical values, in this case, binary state income tax status:

A tile grid map in the shape of the USA is composed of pink and gray squares representing states. Pink states have state income tax while gray states do not.
Tile grid map of state income tax status (by author)
While the map uses two categories (state tax or no state tax), it can be easily adapted for multiple categories, such as those used in the previous abortion access map. The tile template and associated data columns are imported programmatically from the same GIST that holds the template file.

Here’s the full code. I’ll discuss it in more detail in the following sections.

In [7]:
!which python  # Check the Python environment being used
!pip show nbformat  # Check if nbformat is installed and its version
!pip install --upgrade nbformat  # Install or upgrade nbformat

/Users/ulrike_imac_air/projects/code/code_env/bin/python
Name: nbformat
Version: 5.10.4
Summary: The Jupyter Notebook format
Home-page: https://jupyter.org
Author: 
Author-email: Jupyter Development Team <jupyter@googlegroups.com>
License: BSD 3-Clause License

- Copyright (c) 2001-2015, IPython Development Team
- Copyright (c) 2015-, Jupyter Development Team

All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice,
   this list of conditions and the following disclaimer in the documentation
   and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its
   contributors may be used to endorse or promote products de

In [1]:
# Tile Grid Map
# Categorical data example:

import pandas as pd
import plotly.graph_objects as go

# Assign the name of the DataFrame column to plot:
VALUES = 'Tax'

# Assign the plot's title:
TITLE = 'State Income Tax Status'

# Assign the colors for categorical data:
COLOR1 = 'pink'
COLOR2 = 'lightgray'

# Assign colors for tile edge and text:
EDGE_COLOR = 'white'
TEXT_COLOR = 'black'

# Assign Legend Text:
LEGEND_TXT_1 = 'State Income Tax'
LEGEND_TXT_2 = 'No State Income Tax'

# Identify data source:
SOURCE = 'Investopedia.com'

# Load the data:
dataUSA = pd.read_csv('dataUSA.csv')

# Create figure:
fig = go.Figure()

# Assign colors based on a condition:
colors = [COLOR1 if val == 'Yes' else COLOR2 for val in dataUSA[VALUES]]

# Build rectangle shapes for each data point:
shapes = [dict(type='rect',
               x0=x - 0.5, x1=x + 0.5,
               y0=y - 0.5, y1=y + 0.5,
               line=dict(color='white'),
               fillcolor=color,
               layer='below')
          for x, y, color in zip(dataUSA['X'], dataUSA['Y'], colors)]

# Prepare custom data for hover tooltips using a DataFrame slice:
custom_data = dataUSA[['State', VALUES]].values.tolist()

# Add text trace for state abbreviations:
fig.add_trace(go.Scatter(x=dataUSA['X'],
                         y=dataUSA['Y'],
                         mode='text',
                         text=dataUSA['Abbr'],
                         textfont=dict(size=16, 
                                       color=TEXT_COLOR, 
                                       family='Arial'),
                         customdata=custom_data,
                         hovertemplate=('State: %{customdata[0]}<br>' +
                                        'X: %{x}<br>' +
                                        'Y: %{y}<br>' +
                                        'Data: %{customdata[1]}<extra></extra>'),
                         showlegend=False))

# Add legend traces:
fig.add_trace(go.Scatter(x=[None],
                         y=[None],
                         mode='markers',
                         marker=dict(size=15, color=COLOR1, 
                                     symbol='square', 
                                     line=dict(color=COLOR1, width=2)),
                         name=f"<b>{LEGEND_TXT_1}</b>"))
fig.add_trace(go.Scatter(x=[None],
                         y=[None],
                         mode='markers',
                         marker=dict(size=15, color=COLOR2, symbol='square', 
                                     line=dict(color=COLOR2, width=2)),
                         name=f'<b>{LEGEND_TXT_2}</b>'))

# Update figure:
fig.update_layout(shapes=shapes,
                  xaxis=dict(showgrid=False, 
                             zeroline=False, 
                             showticklabels=False),
                  yaxis=dict(showgrid=False,
                             zeroline=False,
                             showticklabels=False,
                             scaleanchor="x",
                             scaleratio=1),
                  plot_bgcolor='white',
                  width=800, height=500,
                  title=dict(text=f'<b>{TITLE}</b>', 
                             font=dict(size=24)),
                  showlegend=True,
                  legend=dict(x=0.95),  # Adjust horizontal position
                  annotations=[dict(x=0,
                                    y=-0.1,
                                    xref='paper',
                                    yref='paper',
                                    text=f'<i>Source: {SOURCE}</i>',
                                    showarrow=False,
                                    font=dict(size=12, color='black'))])

fig.show()

### Continuous Code
The following code example creates a tile grid map of continuous values, in this case, state populations:



In [8]:
# Tile Grid Map
# Continuous data example:

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# Assign the name of DataFrame column to plot:
VALUES = 'Popl'

# Assign plot and legend titles:
TITLE = 'State Populations'
LEGEND_TITLE = 'Population'

# Assign a Plotly colormap 
# See (https://plotly.com/python/builtin-colorscales/):
COLORMAP = 'Blues'

# Assign colors for tile edge and text:
EDGE_COLOR = 'white'
TEXT_COLOR = 'black'

# Identify data source:
SOURCE = 'U.S. Census Bureau'

# Load the CSV file
dataUSA = pd.read_csv('dataUSA.csv')

# Create a figure:
fig = go.Figure()

# Compute normalized population values for interpolating colors:

dataUSA = dataUSA.assign(norm_pop=((dataUSA[VALUES] - dataUSA[VALUES].min()) / 
                                   (dataUSA[VALUES].max() - dataUSA[VALUES].min())))
colors = px.colors.sample_colorscale(COLORMAP, dataUSA['norm_pop'])

# Make square tiles for states:
shapes = [dict(type='rect',
                    x0=x - 0.5, x1=x + 0.5,
                    y0=y - 0.5, y1=y + 0.5,
                    line=dict(color=EDGE_COLOR),
                    fillcolor=color,
                    layer='below')
              for x, y, color in zip(dataUSA['X'], dataUSA['Y'], colors)]
fig.update_layout(shapes=shapes)

# Add the invisible trace for the colorbar first,
# disabling its hover info so it never catches the pointer:
fig.add_trace(go.Scatter(x=dataUSA['X'],
                         y=dataUSA['Y'],
                         mode='markers',
                         marker=dict(colorscale=COLORMAP,
                                     cmin=dataUSA[VALUES].min(),
                                     cmax=dataUSA[VALUES].max(),
                                     color=dataUSA[VALUES],
                                     showscale=True,
                                     colorbar=dict(title=LEGEND_TITLE)),
                         opacity=0,
                         showlegend=False,
                         hoverinfo='skip'))

# Create custom data containing only "State" and "Popl":
custom_data = dataUSA[['State', VALUES]].values.tolist()

# Add a trace with markers + text.
# The invisible markers (set using an RGBA color with 0 alpha) create a 
# hoverable area and the text provides the state abbreviations:
fig.add_trace(go.Scatter(
    x=dataUSA['X'],
    y=dataUSA['Y'],
    mode='markers+text',
    marker=dict(size=20, color='rgba(0, 0, 0, 0)'),
    text=dataUSA['Abbr'],
    textfont=dict(size=16, color=TEXT_COLOR, family='Arial'),
    customdata=custom_data,
    hovertemplate=(
        'State: %{customdata[0]}<br>' +
        f'{LEGEND_TITLE}: ' +
        '%{customdata[1]:,}' +
        '<extra></extra>'),  
    showlegend=False))

# Update figure:
fig.update_layout(xaxis=dict(showgrid=False,
                             zeroline=False,
                             showticklabels=False),
                  yaxis=dict(showgrid=False,
                             zeroline=False,
                             showticklabels=False,
                             scaleanchor='x',
                             scaleratio=1),
                  plot_bgcolor='white',
                  width=800,
                  height=500,
                  title=dict(text=f'<b>{TITLE}</b>',
                             font=dict(size=24)),
                  showlegend=False,
                  annotations=[dict(x=0,
                                    y=-0.1,
                                    xref='paper',
                                    yref='paper',
                                    text=f'<i>Source: {SOURCE}</i>',
                                    showarrow=False,
                                    font=dict(size=12, color='black'))])

fig.show()

### Hexagonal Tile Grid Maps
Using hexagons in place of squares produces a hexagonal tile grid map:

Theoretically, you can achieve better state boundary relationships with hexagons versus square tiles. I don’t find this beneficial enough to justify the fiddly code for building the hexagons, plus I find the hexagons hard on the eyes. Some organizations love them, however. Here’s an interesting article on hex grids by National Public Radio.

The NPR article shows an alternative way to arrange the state tiles. If you want to mimic their arrangement, you’ll need to change the “X” and “Y” columns in the CSV template.

In the following code, I use the same x-y values for the hexagon map as for the square tile map. The example uses continuous data and produces the population map at the start of this section. It can be easily adapted to use categorical data.



In [ ]:
# Tile Grid Map
# Hexagon example (continuous data):

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

# Assign the name of the DataFrame column to plot:
VALUES = 'Popl'

# Assign plot title and legend title:
TITLE = 'State Populations'
LEGEND_TITLE = 'Population'

# Assign a Plotly colormap 
# (see https://plotly.com/python/builtin-colorscales/): BrBG, PRGn, PIYG, PuOr, RdBu, RdBu
# RdGy, RdYlBu, RdYlGn, Spectral, balance, delta, curl, oxy, Armyrose, Fall, Geyser
# Temps, Tealrose, Tropic, Earth, Picnic, Portland
COLORMAP = 'Earth'

# Assign colors for tile edge and text:
EDGE_COLOR = 'white'
TEXT_COLOR = 'white'

# Identify data source:
SOURCE = 'U.S. Census Bureau'

# Choose a hexagon size:
HEX_SIZE = 0.5

# Define a vertical scaling factor to bring hexagons close together:
Y_SCALE = 0.8

# Load the CSV file:
data = pd.read_csv('dataUSA.csv')

# Make a new DF column for the vertical gap between hexagons:
data['Y_scaled'] = data['Y'] * Y_SCALE

# Compute normalized population values for interpolating colors:
data = data.assign(norm_pop=((data[VALUES] - data[VALUES].min()) / 
                             (data[VALUES].max() - data[VALUES].min())))
colors = px.colors.sample_colorscale(COLORMAP, data['norm_pop'])

# Define a function for hexagon shapes:
def create_hexagon_path(center, size, rotation=np.pi/2):
    """
    Given a center (x, y) and a size (distance from the center to a vertex),
    compute an SVG path string for a regular hexagon rotated by a given angle.
    
    The rotation parameter is in radians. 
    The Default is np.pi/2 for a 90° rotation.
    """
    # Generate 7 angles between 0 and 2π, then add rotation offset:
    angles = np.linspace(0, 2 * np.pi, 7) + rotation
    x_coords = center[0] + size * np.cos(angles)
    y_coords = center[1] + size * np.sin(angles)
    
    # Build the SVG path string: "M x0,y0 L x1,y1 ... Z"
    points = [f"{x},{y}" for x, y in zip(x_coords, y_coords)]
    path = "M " + " L ".join(points) + " Z"
    return path

# Define a function to offset every other row so the hexagons interlock:
def offset_x(row):
    """
    Offset every other row so the hexagons interlock.
    
    For odd rows (when converted to int), add an offset equal to the
    HEX_SIZE (half the width)
    """
    if int(row['Y']) % 2 == 1:
        return row['X'] + HEX_SIZE
    else:
        return row['X']

data['X_offset'] = data.apply(offset_x, axis=1)

# Create hexagon shapes using the new (rotated) hexagon path:
shapes = [dict(type='path',
               # Pass an explicit 90° rotation (np.pi/2) to rotate shapes:
               path=create_hexagon_path((x_off, y), HEX_SIZE, rotation=np.pi/2),
               line=dict(color=EDGE_COLOR),
               fillcolor=color,
               layer='below')
          for x_off, y, color in zip(data['X_offset'], 
                                     data['Y_scaled'], 
                                     colors)]

# Create the Plotly figure and add the hexagon shapes:
fig = go.Figure()
fig.update_layout(shapes=shapes)

# Add the invisible scatter trace for the colorbar:
fig.add_trace(go.Scatter(x=data['X_offset'],
                         y=data['Y_scaled'],
                         mode='markers',
                         marker=dict(colorscale=COLORMAP,
                                     cmin=data[VALUES].min(),
                                     cmax=data[VALUES].max(),
                                     color=data[VALUES],
                                     showscale=True,
                                     colorbar=dict(title=LEGEND_TITLE)),
                         opacity=0,
                         showlegend=False,
                         hoverinfo='skip'))

# Prepare custom data for the hover tooltip:
custom_data = data[['State', VALUES]].values.tolist()

# Add the scatter trace with invisible markers and text labels:
fig.add_trace(go.Scatter(x=data['X_offset'],
                         y=data['Y_scaled'],
                         mode='markers+text',
                         marker=dict(size=20, 
                                     color='rgba(0, 0, 0, 0)'),
                         text=data['Abbr'],
                         textfont=dict(size=16, 
                                       color=TEXT_COLOR, 
                                       family='Arial'),
                        customdata=custom_data,
                        hovertemplate=('State: %{customdata[0]}<br>' +
                                       f'{LEGEND_TITLE}: ' +
                                       '%{customdata[1]:,}' +
                                       '<extra></extra>'),
                        showlegend=False))

# Update the figure layout:
fig.update_layout(xaxis=dict(showgrid=False, 
                             zeroline=False, 
                             showticklabels=False),
                  yaxis=dict(showgrid=False, 
                             zeroline=False, 
                             showticklabels=False,
                             scaleanchor='x', 
                             scaleratio=1),
                  plot_bgcolor='white',
                  width=800,
                  height=500,
                  title=dict(text=f'<b>{TITLE}</b>', 
                             font=dict(size=24)),
                  showlegend=False,
                  annotations=[dict(x=0,
                                    y=-0.1,
                                    xref='paper',
                                    yref='paper',
                                    text=f'<i>Source: {SOURCE}</i>',
                                    showarrow=False,
                                    font=dict(size=12, color='black'))])

fig.show()